In [1]:
# Variables

# Storage account name for the Synapse WS storage account - starts with "synsa"
storage_acct_name = "synsa3uka6z6tqjlmi"
storage_container_name = "workspace"
storage_path_raw = "lab-data/raw/"

# Cosmos DB
cosmos_db_container_metadata = "metadata"

# Synapse linked service pointing to Cosmos DB Analytical Store - this is where we get the source data
synapse_cosmos_db_linked_service = "CosmosDbIoTLab"

StatementMeta(spark8, 2, 3, Finished, Available)

In [2]:
vehicle_metadata_df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", synapse_cosmos_db_linked_service)\
    .option("spark.cosmos.container", cosmos_db_container_metadata)\
    .load()

StatementMeta(spark8, 2, 4, Finished, Available)

In [3]:
print(vehicle_metadata_df.count())

vehicle_metadata_df.printSchema()

StatementMeta(spark8, 2, 5, Finished, Available)

173707
root
 |-- _rid: string (nullable = true)
 |-- _ts: long (nullable = true)
 |-- id: string (nullable = true)
 |-- _etag: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- entityType: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- lastServiceDate: string (nullable = true)
 |-- batteryAgeDays: long (nullable = true)
 |-- batteryRatedCycles: long (nullable = true)
 |-- lifetimeBatteryCyclesUsed: double (nullable = true)
 |-- averageDailyTripDuration: double (nullable = true)
 |-- batteryFailurePredicted: boolean (nullable = true)
 |-- stateVehicleRegistered: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- description: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- deliveryDueDate: string (nullable = true)
 |-- packages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- timestamp: string (nullable = true)
 |-- tripId: string (nullable = true)
 |-- consignmentId: string (nu

In [4]:
vehicle_metadata_df.createOrReplaceTempView("metadata")

StatementMeta(spark8, 2, 6, Finished, Available)

In [5]:
trips_clean_df = spark.sql("""
    SELECT  vin, 
            to_utc_timestamp(tripEnded, \"yyyy-MM-dd'T'HH:mm:ss.SSSX'Z'\") as tripEnded, 
            to_utc_timestamp(tripStarted, \"yyyy-MM-dd'T'HH:mm:ss.SSSX'Z'\") as tripStarted, 
            ((unix_timestamp(to_utc_timestamp(tripEnded, \"yyyy-MM-dd'T'HH:mm:ss.SSSX'Z'\")) - 
                unix_timestamp(to_utc_timestamp(tripStarted, \"yyyy-MM-dd'T'HH:mm:ss.SSSX'Z'\")))/60.0) as tripDurationMinutes
    FROM metadata
    WHERE entityType = 'Trip' AND status = 'Completed'
    """)

trips_clean_df.createOrReplaceTempView("trips_clean")

StatementMeta(spark8, 2, 7, Finished, Available)

In [6]:
print(trips_clean_df.count())

trips_clean_df.printSchema()

StatementMeta(spark8, 2, 8, Finished, Available)

110
root
 |-- vin: string (nullable = true)
 |-- tripEnded: timestamp (nullable = true)
 |-- tripStarted: timestamp (nullable = true)
 |-- tripDurationMinutes: decimal(27,6) (nullable = true)

In [7]:
vehicles_raw_df = spark.sql("""
    SELECT vin, batteryAgeDays, batteryRatedCycles, lifetimeBatteryCyclesUsed 
    FROM metadata 
    WHERE entityType ='Vehicle'
    """)

vehicles_raw_df.createOrReplaceTempView("vehicles_raw")

StatementMeta(spark8, 2, 9, Finished, Available)

In [8]:
print(vehicles_raw_df.count())

vehicles_raw_df.printSchema()

StatementMeta(spark8, 2, 10, Finished, Available)

1000
root
 |-- vin: string (nullable = true)
 |-- batteryAgeDays: long (nullable = true)
 |-- batteryRatedCycles: long (nullable = true)
 |-- lifetimeBatteryCyclesUsed: double (nullable = true)

In [9]:
batch_df = spark.sql("""
    SELECT  v.vin as vin, 
            to_date(t.tripEnded, 'yyyy-MM-dd') as tripEnded, 
            t.tripDurationMinutes, 
            v.batteryAgeDays, 
            v.batteryRatedCycles, 
            v.lifetimeBatteryCyclesUsed,
            CASE
                WHEN ( v.lifetimeBatteryCyclesUsed + (30 * v.lifetimeBatteryCyclesUsed / v.batteryAgeDays) > v.batteryRatedCycles ) THEN 1
                ELSE 0
            END as maint_needed
    FROM    vehicles_raw v 
    INNER JOIN trips_clean t 
        ON v.vin = t.vin
    """)

StatementMeta(spark8, 2, 11, Finished, Available)

In [10]:
print(batch_df.count())

batch_df.printSchema()

StatementMeta(spark8, 2, 12, Finished, Available)

110
root
 |-- vin: string (nullable = true)
 |-- tripEnded: date (nullable = true)
 |-- tripDurationMinutes: decimal(27,6) (nullable = true)
 |-- batteryAgeDays: long (nullable = true)
 |-- batteryRatedCycles: long (nullable = true)
 |-- lifetimeBatteryCyclesUsed: double (nullable = true)
 |-- maint_needed: integer (nullable = false)

In [11]:
batch_df.show()

StatementMeta(spark8, 2, 13, Finished, Available)

+-----------------+----------+-------------------+--------------+------------------+-------------------------+------------+
|              vin| tripEnded|tripDurationMinutes|batteryAgeDays|batteryRatedCycles|lifetimeBatteryCyclesUsed|maint_needed|
+-----------------+----------+-------------------+--------------+------------------+-------------------------+------------+
|T8DNDN5UDCWL7M72H|2020-07-09|          19.750000|           112|               200|        12.56852760294626|           0|
|0ZGVI20GIS84M1B4D|2020-07-09|          33.433333|           664|               200|        73.11092063018555|           0|
|9K3NPUOHFCGGMDO9G|2020-07-09|          20.816667|           275|               200|       29.630602915264014|           0|
|V5V483U7H0713ZAQ0|2020-07-09|          23.866667|           814|               200|       103.54608638921663|           0|
|ZUM234INX6MPOJ1D6|2020-07-09|          36.950000|          1071|               200|       123.67412813528725|           0|
|5FY0WL5

In [12]:
sa_uri = "abfss://workspace@" + storage_acct_name + ".dfs.core.windows.net/" + storage_path_raw

batch_df.write.parquet(sa_uri)

StatementMeta(spark8, 2, 14, Finished, Available)